In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import SparkSession
import folium
import shapely.wkt
import json
from geopy.distance import geodesic


In [2]:
spark = SparkSession.builder.appName("example").getOrCreate()
df_spark = spark.read.format("parquet").load("data")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/13 11:43:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_pandas = df_spark.toPandas()

24/12/13 11:43:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=17754Kb max_used=17754Kb free=113317Kb
 bounds [0x000000010d9f8000, 0x000000010eb68000, 0x00000001159f8000]
 total_blobs=6915 nmethods=5967 adapters=861
 compilation: disabled (not enough contiguous free space left)


In [4]:
filtered_df = df_pandas[df_pandas["provider_routing_points"].apply(lambda x: len(x) > 1)]
filtered_df = filtered_df[filtered_df["reference_routing_points"].apply(lambda x: len(x) > 1)]

In [5]:
filtered_df.head()

,api_poi_id,reference_id,matching_run_id,release_version,provider_id,enduser_category,category_id_tt,category_name,reference_translated_cat_id,providers_translated_cat_id,...,region,reference_routing_points,country,provider_response,provider_routing_points,rpav_matching,rpav_matching_run_id,query_lat,query_lon,sample_run_id
24209,76397748250,ChIJESXsBKbGDQ0RKwPBeraQvz8,20240209-22044310,24060.000 OV,om,Health Care,7321,Hospital/Polyclinic,7321,7321,...,"{""country"": ""ESP"", ""a1_admin_code"": null, ""adm...","[[36.6994737, -6.146485999999999], [36.6986097...",ESP,"{""responses"" : [{""id"" : 83436245484, ""lat"" : 3...","[[36.698055267333984, -6.14805269241333], [36....","(([[210.68105607709813, 219.41479492559054], [...",20240216-15544023,None,None,None
24260,83576632843,ChIJIYFgZbYhQg0ROV8VlcrGG-c,20240209-22044310,24060.000 OV,om,Shopping,7373,Shopping Center,7373,7373,...,"{""country"": ""ESP"", ""a1_admin_code"": null, ""adm...","[[40.2709444, -3.6949843], [40.2711297, -3.697...",ESP,"{""responses"" : [{""id"" : 83381566541, ""lat"" : 4...","[[40.27122116088867, -3.6974942684173584], [40...","(([[215.16107076272885, 328.40148637080665], [...",20240216-15544023,None,None,None
24362,49423943060,ChIJedXaFdwkQg0R2-BMyAo3-cs,20240209-22044310,24060.000 OV,om,Shopping,7373,Shopping Center,7373,9361,...,"{""country"": ""ESP"", ""a1_admin_code"": null, ""adm...","[[40.3680552, -3.6011447], [40.3708911, -3.597...",ESP,"{""responses"" : [{""id"" : 82009320719, ""lat"" : 4...","[[40.36901092529297, -3.5990352630615234], [40...","(([[207.91989440607958, 308.42814944118555], [...",20240216-15544023,None,None,None
24377,83795330623,ChIJE9JYe6Y3Yg0RzWsSQZQSQsw,20240209-22044310,24060.000 OV,om,Health Care,7321,Hospital/Polyclinic,7321,9663,...,"{""country"": ""ESP"", ""a1_admin_code"": null, ""adm...","[[38.35291180000001, -0.4765705], [38.3530048,...",ESP,"{""responses"" : [{""id"" : 82585028023, ""lat"" : 3...","[[38.35302734375, -0.4763830900192261], [38.35...","(([[20.787770245465314, 9.297391407258868], [2...",20240216-15544023,None,None,None
24383,84154595663,ChIJd_z05hGjpBIRl_5UAkf7vNo,20240209-22044310,24060.000 OV,om,Shopping,7373,Shopping Center,7373,7373,...,"{""country"": ""ESP"", ""a1_admin_code"": null, ""adm...","[[41.3896711, 2.198145], [41.39081119999999, 2...",ESP,"{""responses"" : [{""id"" : 83217614967, ""lat"" : 4...","[[41.3901481628418, 2.1975386142730713], [41.3...","(([[73.30006070245875, 70.26047809655567], [10...",20240216-15544023,None,None,None


In [6]:
for index, row in df_pandas.iterrows():
    if len(row['reference_routing_points']) > 1 & len(row['provider_routing_points']) > 2:
        print(f"Place: {index}, Routing Points: {len(row['reference_routing_points'])} and {len(row['provider_routing_points'])}")

In [7]:
#Features to add
# circle around poi loc if they rp and poi are the same
# polygon for poi geometry
# show google maps results for some examples

In [8]:
def provider_char_d(res):
    return res['responses'][0]['lat'], res['responses'][0]['lon']

def provider_latlon_(res):
    return res['responses'][0]['lat'], res['responses'][0]['lon']

def max_distance(centroid, markers):
    max_dist = 0
    # Compare every pair of markers
    for i in markers:
            dist = geodesic(centroid, i).km
            if dist > max_dist:
                max_dist = dist

    return max_dist*1000  # Return the distance in meters

# Function to calculate bounding box from a point and a distance (in meters)
def calculate_bounds(lat, lon, distance_meters):
    # Approximate the distance in degrees (1 degree of latitude is about 111 km)
    lat_offset = distance_meters / 111320  # 1 degree of latitude ~ 111320 meters
    lon_offset = distance_meters / (40008000 * (1 / 360)) * (1 / (111320 * 2))  # Roughly adjust for longitude

    # Calculate the bounding box
    min_lat = lat - lat_offset
    max_lat = lat + lat_offset
    min_lon = lon - lon_offset
    max_lon = lon + lon_offset

    return [[min_lat, min_lon], [max_lat, max_lon]]

In [9]:
def mean_distance_to_routing_points(poi_latlon, routing_points):
    distances = []
    for rp in routing_points:
        distance = geodesic(poi_latlon, rp).meters
        distances.append(distance)
    mean_distance = sum(distances) / len(distances) if distances else 0
    return mean_distance

In [10]:
row = df_pandas.iloc[24377]

rppa = row['rpav_matching']['fields']['rppa']
poi_characteristic_distance = row['rpav_matching']['fields']['poi_characteristic_distance']
assignation = row['rpav_matching']['fields']['assignation']
provider_routing_points = row["provider_routing_points"]
reference_latlon = (float(row['ref_lat']), float(row['ref_lon']))
provider_latlon = provider_latlon_(json.loads(row['provider_response']))  
reference_routing_points =  row["reference_routing_points"]

map_centroid = reference_latlon
width, height = 800, 600 
m = folium.Map(location=map_centroid, zoom_start=17, tiles='openstreetmap', width=width, height=height)

folium.Marker(location= reference_latlon, icon=folium.Icon(color='black', icon = "")).add_to(m)
folium.Marker(location= provider_latlon, icon=folium.Icon(color='red', icon = "")).add_to(m)

markers = [reference_latlon, provider_latlon]

for rp in provider_routing_points:
    folium.Circle(location=rp, radius=0.7*poi_characteristic_distance, color="red").add_to(m)
    folium.CircleMarker(location=rp, radius=4, color="red", fill=False, fill_color="red", fill_opacity=1).add_to(m)

    markers.append(rp)
    folium.PolyLine(locations=[rp, provider_latlon], color="red", weight=2, opacity=1, dashArray="5, 5").add_to(m)

if rppa > 0:
    for asign in assignation:
        if geodesic(reference_routing_points[asign[0]], provider_routing_points[asign[1]]).m < 0.7*poi_characteristic_distance:
            folium.PolyLine(locations=[reference_routing_points[asign[0]], provider_routing_points[asign[1]]], color="green", weight=4, opacity=1).add_to(m)

for rp in reference_routing_points:
    folium.CircleMarker(location=rp, radius=4, color="black", fill=True, fill_color="black", fill_opacity=1).add_to(m)
    markers.append(rp)
    folium.PolyLine(locations=[rp, reference_latlon], color="black", weight=2, opacity=1, dashArray="5, 5").add_to(m)


    
bounds = calculate_bounds(map_centroid[0], map_centroid[1], 1.5*max_distance(map_centroid, markers))
m.fit_bounds(bounds)

m

In [11]:
assignation

[[0, 1], [1, 0]]

In [12]:
rppa

1.0

In [13]:
new_df = df_pandas[['name', 'provider_routing_points', 'reference_routing_points']]


In [14]:
new_df

,name,provider_routing_points,reference_routing_points
0,Siam Park,"[[28.07150650024414, -16.726531982421875]]","[[28.0693128, -16.7243125], [28.0714587, -16.7..."
1,Aqualand Costa Adeje,"[[28.079387664794922, -16.72748565673828]]","[[28.0806669, -16.727891]]"
2,Playa Punta Gorda,"[[28.11623191833496, -15.705873489379883]]","[[28.1058227, -15.709638]]"
3,Plaza Teresa de Bolívar,"[[28.05979347229004, -15.546832084655762]]","[[28.0598298, -15.5467797]]"
4,Playa de Salinetas,"[[27.982881546020508, -15.378780364990234]]","[[27.9832465, -15.3788078], [27.982, -15.37875..."
...,...,...,...
35087,Restaurante Brisas de Abades,"[[28.142688751220703, -16.44021224975586]]","[[28.142674, -16.4401896]]"
35088,Casa Wimba,"[[40.13690948486328, -4.690561294555664]]","[[40.1369812, -4.6903822]]"
35089,Restaurante Apolo XV,"[[37.50648880004883, -5.378880500793457]]","[[37.506377, -5.3787141]]"
35090,Los Rastrojos Hotel Restaurante,"[[41.683380126953125, -3.634113311767578]]","[[41.6834663, -3.6341696]]"
